In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.fft import fft, ifft,fftfreq
from matplotlib import animation
from scipy.optimize import curve_fit
#plt.rc('animation', html='jshtml')
plt.style.use('ggplot')
%matplotlib qt

Der indlæses startsbetingelser jævnfør dokumentet:

In [2]:
N = 2**13 #antal punkter i xgrid og pgrid
dt = .01
timesteps = 3000
x=np.linspace(0,1000,N)
dx = (max(x)-min(x))/(N-1)
p = fftfreq(N, dx)*2*np.pi #Ved at lave p-griddet på den her måde skal man ikke gå fra psi til f osv.

In [3]:
k0 = 5 #startimpuls

Der genereres en gaussisk startbølgefunktionen der passer nogenlunde til det xgrid, der er blevet fastlagt ved dx og N. Også et potentiale.

In [4]:
a=10
psi_0 = np.exp(-(x-150)**2/a**2)*np.exp(1.j*k0*x)
psi_0 = psi_0/np.sqrt(np.trapz(abs(psi_0)**2,x))



#psi_0 = norm.pdf(x, np.mean(x)-40, 20) * np.exp(-1j*k0*x) #gaussisk starbølgefunktion
#V = lambda x: .01*(x-110)**2 #polynomielt potentiale

Va = np.zeros_like(x)
#Va[int(len(x)*0.52):int(len(x)*0.54)] = 5
Va = 12.5*np.exp(-(x-200)**2/4./1**2)
V = lambda x: Va

Her defineres de deloperationer, der vil blive anvendt i simulationen:

In [5]:
def t_operator(f): #Anvend T-operator delen fra lign 24
    return f * np.exp(-1.j*p**2/2 *dt/2)

def v_operator(f): #Anvend V-operator delen fra lign 24
    return f * np.exp(-1.j*Va*dt)

Endeligt køres koden igennem pr. tidsskridt. Har prøvet at markere den logiske fremgang i kode-kommentarene:

In [6]:
psis = []
phis = []

psi = psi_0

for t in range(timesteps):
    if t % 20 == 0: #Gem kun per. 20. tidsskridt (så går animationen lidt hurtigere)
        psis.append(psi)
    if t % 20 == 0:
        phis.append(fft(psi)/np.sqrt(np.trapz(abs(fft(psi))**2,p)))
    f = psi
    f = t_operator(fft(f)) # f → T_operator(ftilde)
    f = v_operator(ifft(f)) # ftilde → V_operator(f)
    f = t_operator(fft(f)) # f → T_operator(ftilde)
    f = ifft(f) # ftilde → f
    psi = f
    psi = psi/np.sqrt(np.trapz(abs(psi)**2,x))

Plot og animer potentiale og bølgefunktion:

In [7]:
fig, ax = plt.subplots(1,1,tight_layout=True)
psi_plot, = ax.plot([],[],lw=.4,color='k',label=r'$|\Psi|^2$')
#ax.set_ylim(0,0.1)
ax.legend(frameon=False,fontsize=14)
#ax.set_xticklabels([])
#ax.set_yticklabels([])
ax.set_xlabel('x')
ax.set_ylabel(r'Relative sandsynlighedsdensitet $\rho$')
ax.set_title('Kvantemekanisk potentialetunnelering')
ax.grid(b=None)
ax2 = ax.twinx()
#ax2.set_ylim(0,400)
ax2.set_yticklabels([])
ax2.show_yticks = False
ax2.plot(x,Va,color='red',label='V(x)',lw=.4)
ax2.fill_between(x,Va,color='k',alpha=0.5)
ax2.legend(frameon=False,loc='upper left',fontsize=14)
ax2.set_ylabel('Potentiale')
ax2.grid(b=None)
ax.set_xlim(0,x[-1])
itext = ax.text(600,0.7,s='',fontsize=12)
left_side_text = ax.text(10,0.2,s='',fontsize=12)
right_side_text = ax.text(600,0.2,s='',fontsize=12)
def update(i):
    psi_plot.set_data(x,np.abs(psis[i])**2)
    left_side_text.set_text('$\int$ = ' +str(np.round(np.trapz(x[0:1638],np.abs(psis[i][0:1638])**2)/np.trapz(x,np.abs(psis[i])**2),decimals=2)))
    right_side_text.set_text('$\int$ = ' + str(np.round(np.trapz(x[1639:-1],np.abs(psis[i][1639:-1])**2)/np.trapz(x,np.abs(psis[i])**2),decimals=2)))
    itext.set_text('t=' + str(i))
    return [psi_plot,left_side_text,right_side_text,itext]

anim = animation.FuncAnimation(fig,
                               update,
                               frames=int(timesteps/10-1),
                               interval=5)
anim

#### Undersøgelse af energibevarelse

In [8]:
normalized_psis = [np.abs(psis[i])/np.sqrt(np.trapz(np.abs(psis[i])**2,x)) for i in range(len(psis))]
integrals = [np.trapz(np.abs(psis[i])**2,x) for i in range(len(psis))]

normalized_phis = [np.abs(phis[i])/np.sqrt(np.trapz(np.abs(phis[i])**2,p)) for i in range(len(phis))]
integrals_phis = [np.trapz(np.abs(phis[i])**2,p) for i in range(len(phis))]

In [10]:
Kin = [p**2 /2 * np.array(normalized_phis[i]) for i in range(len(phis))]
Pot = [V(x) * np.array(normalized_psis[i]) for i in range(len(psis))]
E_kins = [np.trapz(np.conjugate(normalized_phis[i])*Kin[i],p) for i in range(len(phis))]
E_pot = [np.trapz(np.conjugate(normalized_psis[i])*Pot[i],x) for i in range(len(psis))]
Es = np.array(E_kins) + np.array(E_pot)

In [14]:
efig, eax = plt.subplots(1,1)
eax.set_title('Energien igennem simulationen')
eax.set_xlabel('Tid')
eax.set_ylabel('Energi')
eax.plot(np.array(E_kins),label='Kinetiske energi')
eax.plot(np.array(E_pot),label='Potentielle energi')
eax.plot(np.array(Es),label='Samlede energi')
eax.legend(frameon=False,fontsize=12)

In [12]:
Energies = np.array([12.51,11.525,12.01,13.01,11.77,12.26,12.76])
Transmissions = np.array([0.51,0.13,0.28,0.73,0.19,0.39,0.63])